In [0]:
%sql
select * from dev.bronze.sales 

In [0]:
df=spark.read.table("dev.bronze.sales")
df_final=df.dropDuplicates().dropna().drop("ingestion_date")
df_final.write.mode("overwrite").saveAsTable("dev.silver.sales")

In [0]:
%sql
create or replace table dev.silver.products_scd (product_id int, product_name string, product_category string, product_price)

In [0]:
%sql

MERGE INTO dev.silver.customers_scd AS target
USING dev.bronze.product AS source
ON target.product_id = source.product_id
WHEN MATCHED AND source.operation = 'DELETE' THEN DELETE
WHEN MATCHED AND source.operation = 'UPDATE' THEN UPDATE SET *
WHEN NOT MATCHED AND source.operation = 'INSERT' THEN
  INSERT (product_id, product_name, product_category, product_price)
  VALUES (source.product_id, source.product_name, source.product_category, source.product_price)

In [0]:
%sql
select * from dev.silver.products_scd

In [0]:
%sql
-- describe dev.bronze.customers;

create or replace table dev.silver.customers_scd2( customer_id int, customer_name string, customer_email string, customer_city string, customer_state string)

In [0]:
%sql
select * from dev.bronze.customers

In [0]:
%sql
MERGE INTO dev.silver.customers_scd2 AS target
USING (
  SELECT *
  FROM dev.bronze.customers
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sequenceNum DESC) = 1
) AS source
ON target.customer_id = source.customer_id -- Corrected the join key

-- Condition 1: The latest operation for a matched customer was DELETE
WHEN MATCHED AND source.operation = 'DELETE' THEN
  DELETE

-- Condition 2: The latest operation for a matched customer was UPDATE (or a re-INSERT)
WHEN MATCHED AND source.operation IN ('UPDATE', 'INSERT') THEN
  UPDATE SET
    target.customer_name = source.customer_name,
    target.customer_email = source.customer_email,
    target.customer_city = source.customer_city,
    target.customer_state = source.customer_state

-- Condition 3: The customer does not exist in the silver table, and the operation is INSERT
WHEN NOT MATCHED AND source.operation = 'INSERT' THEN
  -- This correctly inserts brand new customers like customer_id 6
  INSERT (
    customer_id,
    customer_name,
    customer_email,
    customer_city,
    customer_state
  )
  VALUES (
    source.customer_id,
    source.customer_name,
    source.customer_email,
    source.customer_city,
    source.customer_state
  )

In [0]:
%sql
select * from dev.silver.customers_scd2